In [1]:
import os
import sys
import h5py
import glob
import time
import numpy as np
import pandas as pd
import pickle as pickle
from pytz import timezone
import astropy.io.fits as pf
from astropy.time import Time
import matplotlib.pyplot as plt
from skyfield.api import load, wgs84, EarthSatellite

## V2: switching the way times are defined

In [2]:
### FUNCTIONS NEEDED TO TRANSFORM PIXEL COORDS TO LOOK AT FOV

def world2pix(lst, ra, dec, jd=None, nx=4008, ny=2672, margin=50):
    xwcs, ywcs = world2wcs(wcspars, ra, dec, lst, jd) # Compute wcs-only pixel coordinates
    mask = np.isfinite(xwcs) & clip_rectangle(xwcs, ywcs, nx, ny, margin) # Remove coordinates not within the margins
    xwcs, ywcs = xwcs[mask], ywcs[mask]        
    xpix, ypix = wcs2pix(polpars, xwcs, ywcs) # Convert to actual pixel coordinates.
    return xpix, ypix, mask


def clip_rectangle(x, y, nx, ny, margin):
    maskx = (x > margin) & (x < (nx - margin))
    masky = (y > margin) & (y < (ny - margin))
    return maskx & masky


def world2wcs(wcspars, ra, dec, lst=None, jd=None):
    """ Convert world coordinates to WCS-only coordinates. """    
    if jd is not None:
        ra, dec = j2000_to_equinox(ra, dec, jd)
    w = create_wcs(wcspars, lst)
    xwcs, ywcs = w.wcs_world2pix(ra, dec, 0)
    return xwcs, ywcs


def create_wcs(wcspars, lst=None):
    """ Create and astropy WCS instance from a dictionary of parameters. """  
    from astropy import wcs
    
    if lst is not None:
        ra0, dec0 = wcspars['crval']
        ha0 = ra2ha(ra0, wcspars['lst'])
        ra0 = ha2ra(ha0, lst)
        crval = np.array([ra0, dec0])   
    else:
        crval = wcspars['crval']
        
    w = wcs.WCS(naxis=2)
    w.wcs.crpix = wcspars['crpix']
    w.wcs.cdelt = wcspars['cdelt']
    w.wcs.crval = crval
    w.wcs.ctype = ["RA---TAN", "DEC--TAN"]
    w.wcs.pc = wcspars['pc']
    return w


def ra2ha(ra, lst):
    """ Convert Right Ascension to Hour Angle. """
    ha = np.mod(lst*15. - ra, 360.)
    return ha

def ha2ra(ha, lst):
    """ Convert Hour Angle to Right Ascension. """    
    ra = np.mod(lst*15. - ha, 360.)
    return ra


def j2000_to_equinox(ra, dec, jd):
    from astropy.time import Time
    from astropy.coordinates import SkyCoord, FK5
    t = Time(jd, scale='utc', format='jd') 
    t.format = 'jyear_str'
    gc = SkyCoord(ra, dec, frame='fk5', unit='deg', equinox='J2000')    
    gc = gc.transform_to(FK5(equinox=t))    
    return gc.ra.value, gc.dec.value


def wcs2pix(polpars, xwcs, ywcs):
    """ Convert WCS-only coordinates to pixel coordinates. """     
    dx, dy = leg2d_eval(xwcs, ywcs, polpars['x_wcs2pix'], polpars['y_wcs2pix'], 
                        polpars['order'], polpars['nx'], polpars['ny'])    
    xpix, ypix = xwcs + dx, ywcs + dy
    return xpix, ypix


def leg2d_eval(x, y, a, b, order=6, nx=4008., ny=2672.):
    """ Evaluate 2-D polynomials. """
    mat, idx1, idx2 = leg2d_mat(x, y, order, nx, ny)
    xres = np.dot(mat, a[idx1, idx2])
    yres = np.dot(mat, b[idx1, idx2])
    return xres, yres  


def leg2d_mat(x, y, order=6, nx=4008., ny=2672.):
    """ Create a matrix for fitting 2-D polynomials of the given order. """ 
    from numpy.polynomial import legendre
    x = np.array(x)
    y = np.array(y)
    mat1 = legendre.legvander(2*(x/nx) - 1, order+1)
    mat2 = legendre.legvander(2*(y/ny) - 1, order+1)
        
    idx1, idx2 = np.indices((order+1, order+1))
    mask = ((idx1 + idx2) <= order)
    idx1, idx2 = idx1[mask], idx2[mask]        
        
    mat = mat1[:,idx1]*mat2[:,idx2]
    return mat, idx1, idx2

In [3]:
def reduce_tles(camid):

    # Load TLEs for all passages
    satfiles = '../test_data/3leComplete.txt'
    with open(satfiles) as f:
        all_tles = f.readlines()
        f.close()   

    # Split TLE list into individual lists for each TLE
    all_tles = [i.strip() for i in all_tles]
    tles = [all_tles[x:x+3] for x in range(0, len(all_tles), 3)]

    # Reduce TLEs to Starlink only
    starlink_tles = []
    for tle in tles:
        if "STARLINK" in tle[0]:
            starlink_tles.append(tle)

    # Obtain satellite passages
    passed_sats = pd.read_pickle(f'../test_data/passages/passed_satellites_20221023{camid}.p')

    # Find any Starlink TLEs in the passages
    idx = []
    flatlist = np.asarray(starlink_tles).flatten()
    for key in passed_sats.keys():
        line1 = passed_sats[key]['TLE line1'].strip()
        i = np.where(flatlist == line1)[0] 
        if i.size > 0:
            idx.append(i[0] - 1) #appending the name of the starlink sat

    # Now have indices for the flattened Starlink TLE list --> divide by 3 to get indices for the original list
    orig_idx = [int(x/3) for x in idx]
    passed_tles = [starlink_tles[i] for i in orig_idx]

    # Remove 0 labeling of first line of TLE because that's the proper format
    for tle in passed_tles:
        tle[0] = tle[0][2:]
         
    #print(f'Number of satellites recorded for this day: {len(all_tles)}')
    #print(f'Number of them that were Starlinks: {len(starlink_tles)}')
    #print(f'Number of Starlinks that passed MASCARA: {len(passed_tles)}')
    
    return passed_tles

In [4]:
# starlinks = reduce_tles('LSC')

In [5]:
def image_data(camid):
    
    images = glob.glob(f'../test_data/diff_images/{camid}/diff_*.fits.gz')

    imgdata = {}
    for img in images:
        header = pf.getheader(img) 
        lstseq = img[-19:-11]
        midlst = header['MIDLST']
        midJD = header['MIDJD']
        nx = header['XSIZE']    
        ny = header['YSIZE']

        imgdata[lstseq] = {}
        imgdata[lstseq]['midLST'] = midlst
        imgdata[lstseq]['midJD']  = midJD
        imgdata[lstseq]['nx'] = nx
        imgdata[lstseq]['ny'] = ny
    
    return imgdata

In [6]:
# imgdata = image_data('LSC')

In [7]:
def passage_check(midJD, tles, passages):
        
    idx_reduced = []
    sats = passages[midJD].keys()
    
    satnums = []
    for tle in tles:
        satnums.append(tle[1].split()[1])

    # Cross-reference
    for i, sat in enumerate(sats):
        if sat in satnums:
            idx = satnums.index(sat)
            idx_reduced.append(idx)
  
    return idx_reduced

In [8]:
def get_siteinfo():
    confdir = '../fotos-python3/bringfiles/siteinfo.dat'
    dtype = [('sitename', '|U20'), ('lat', 'float32'), ('lon', 'float32'), ('height', 'float32'), ('ID', '|U2')]
    siteinfo = np.genfromtxt(confdir, dtype=dtype)   
    mask = siteinfo['ID'] == 'LS'
    siteinfo = siteinfo[mask]
    return siteinfo

In [9]:
### Get timerange of all images ---- changing

def image_timerange(imgdata):
    ts = load.timescale()
    dates = []
    for lst in list(imgdata):
        dates.append(imgdata[lst]['midJD'])

    oldest = min(dates)
    newest = max(dates)

    t_old = Time(oldest, format='jd')
    t_new = Time(newest, format='jd')

    timerange = np.linspace(t_old, t_new, 150, endpoint=True) # every ~3mins

    return ts.from_astropy(timerange)

In [10]:
def check_illumination(imgdata, midJD, timerange, sat):

    # Check when satellite is illuminated 
    timerange = image_timerange(imgdata)
    sunlit = sat.at(timerange).is_sunlit(eph)

    # Obtain the indices of the first and last True element for each sequence of True elements 
    sunlit_idx = []
    start_idx = None
    for i, elem in enumerate(sunlit):
        if elem:
            # Checking if the start of a sequence of True elements
            if start_idx is None:
                start_idx = i
        else:
            # Checking if a sequence is just ended
            if start_idx is not None:
                sunlit_idx.append((start_idx, i-1)) # -1 because we're at the first False element after True sequence
                start_idx = None

    # If the last element is True, we need to append its index as well
    if start_idx is not None:
        sunlit_idx.append((start_idx, len(sunlit)-1))
    
    # Getting the times from the indices
    sunlit_times = []
    for idx in sunlit_idx:
        sunlit_times.append([timerange[idx[0]].to_astropy().value, timerange[idx[1]].to_astropy().value])
        #sunlit_times.append([timerange.tt[idx[0]], timerange.tt[idx[1]]])

    # Check if midJD is within each period and flag it if so
    illuminated = False
    for sunlitrng in sunlit_times:
        if midJD >= (sunlitrng[0] - 1/86400) and midJD <= (sunlitrng[1] + 1/86400): #adding a bumper of one second
            illuminated = True
            if illuminated:
                break

    return illuminated

In [63]:
### For Jupyter Notebook playaround, will use LSC and LSE only

### THERE ARE A NUMBER OF THINGS WE DON'T WANT TO REPEAT IN THE MAIN LOOP

# DE421 planetary ephemeris (to get position of Sun)
eph = load('de421.bsp')

def check_each_camid():
    imgdata = []
    times = []
    fasts = []
    psgs = []
    tles = []
    camids = ['LSC','LSE']
    
    print('Collecting image information\n')
    for camid in camids:
        d = image_data(camid)
        imgdata.append(d)
        times.append(image_timerange(d))
        fasts.append(h5py.File(f"../fast_20221023{camid}.hdf5", "r"))
        psgs.append(pd.read_pickle(f"../test_data/passages/passages_20221023{camid}.p"))
        tles.append(reduce_tles(camid))
    
    # Define observer
    site = get_siteinfo()
    mascara = wgs84.latlon(latitude_degrees=site[0][1], longitude_degrees=site[0][2], elevation_m=site[0][3])
    
    print('Beginning Loop\n')
    result = []
    for seq in fasts[0]['station']['lstseq'][:]: 
        lstseq = str(seq)

        for data, timerange, camid, fast, starlinks, passages in zip(
            imgdata, times, camids, fasts, tles, psgs):

            if lstseq not in data.keys():
                #print(f'{lstseq} not in directory')
                continue #this is only needed if not going through ALL images!

            # Set time of image
            midJD = data[lstseq]['midJD']
            ts = load.timescale()
            t = Time(midJD, format='jd')

            # Obtain passages
            idx_reduced = passage_check(midJD, starlinks, passages)

            if idx_reduced is None:
                print(f'{lstseq}: No starlinks passing MASCARA')
                continue

            else:
                print(f'There are {len(idx_reduced)} Starlinks passing overhead {camid} for LSTSEQ={lstseq}')
                # Sun must be low enough below the horizon, otherwise data is not good enough
                observer = mascara + eph['earth']
                sun_pos = observer.at(ts.from_astropy(t)).observe(eph['sun'])
                sun_alt, sun_az, sun_dist = sun_pos.apparent().altaz()
                
                if sun_alt.degrees <= -18.:
                    # Attaining the astrometric solution (depends on FAST file and LSTseq)
                    astro = np.where((fast['astrometry/lstseq'][:] // 50) == (int(lstseq) // 50))[0][0]
                    order = fast['astrometry/x_wcs2pix'][astro].shape[0]-1
                    lst = fast['station/lst'][np.where(fast['station/lstseq'][:]==(fast['astrometry/lstseq'][astro]))[0][0]]
                    nx = data[lstseq]['nx']
                    ny = data[lstseq]['ny']

                    wcspars = { 'crval' : fast['astrometry/crval'][astro].copy(),
                                'crpix' : fast['astrometry/crpix'][astro].copy(),
                                'cdelt' : [0.02148591731740587,0.02148591731740587],
                                'pc'    : fast['astrometry/pc'][astro].copy(),
                                'lst'   : lst }

                    polpars = { 'x_wcs2pix' : fast['astrometry/x_wcs2pix'][astro].copy(),
                                'y_wcs2pix' : fast['astrometry/y_wcs2pix'][astro].copy(),
                                'x_pix2wcs' : fast['astrometry/x_pix2wcs'][astro].copy(),
                                'y_pix2wcs' : fast['astrometry/y_pix2wcs'][astro].copy(),
                                'nx'    : nx,
                                'ny'    : ny,
                                'order' : order }

                    midLST = data[lstseq]['midLST']

                    for idx in idx_reduced:
                        line1 = starlinks[idx][0]
                        line2 = starlinks[idx][1]
                        line3 = starlinks[idx][2] 
                        sat = EarthSatellite(line2, line3, line1, ts)

                        diff = sat - mascara
                        topocentric = diff.at(ts.from_astropy(t))
                        alt, az, dist = topocentric.altaz()

                        # Criteria check: if satellite is >20 degrees above the horizon
                        if alt.degrees >= 20:

                            # Criteria check: if satellite is illuminated
                            illuminated = check_illumination(data, midJD, timerange, sat)

                            if illuminated:
                                ra, dec, radec_dist = topocentric.radec() 
                                radeg = ra._degrees
                                dedeg = dec._degrees
                                ra, dec = j2000_to_equinox(radeg, dedeg, midJD)
                                w = create_wcs(wcspars, midLST)
                                xwcs, ywcs = w.wcs_world2pix(ra, dec, 0)
                                mask = np.isfinite(xwcs) & clip_rectangle(xwcs, ywcs, nx, ny, margin=50)

                                if mask:
                                    print(f"At least one Starlink should be in {camid} FOV")
                                    image = lstseq + camid + ' ' + line1 + ' ' + line2[2:8]
                                    result.append(image)
                                    break #so to continue to next LSTSEQ if at least one starlink is found!

                    print('\n')



    for fast in fasts:
        fast.close()
    
#     if len(result) > 0:
#         np.savetxt('../selection_pool.txt', result, fmt='%s')
        
    return result

In [64]:
%%time
result = check_each_camid()


Beginning Loop

There are 2 Starlinks passing overhead LSC for LSTSEQ=48506263


There are 2 Starlinks passing overhead LSC for LSTSEQ=48506377
At least one Starlink should be in LSC FOV
At least one Starlink should be in LSC FOV


There are 6 Starlinks passing overhead LSE for LSTSEQ=48506377


There are 2 Starlinks passing overhead LSC for LSTSEQ=48506907


There are 2 Starlinks passing overhead LSC for LSTSEQ=48507687


There are 5 Starlinks passing overhead LSE for LSTSEQ=48507687


There are 1 Starlinks passing overhead LSC for LSTSEQ=48508580


There are 1 Starlinks passing overhead LSC for LSTSEQ=48509045


There are 1 Starlinks passing overhead LSC for LSTSEQ=48509945


There are 3 Starlinks passing overhead LSC for LSTSEQ=48509980


There are 3 Starlinks passing overhead LSC for LSTSEQ=48510542


There are 2 Starlinks passing overhead LSC for LSTSEQ=48510733


There are 2 Starlinks passing overhead LSC for LSTSEQ=48510920
There are 1 Starlinks passing overhead LSC for LSTSEQ=

In [24]:
result

['48506377LSC STARLINK-2631 48366U', '48506377LSC STARLINK-3641 51788U']

# V3 - recording passages as pickle file

In [68]:
### For Jupyter Notebook playaround, will use LSC and LSE only

### THERE ARE A NUMBER OF THINGS WE DON'T WANT TO REPEAT IN THE MAIN LOOP

# DE421 planetary ephemeris (to get position of Sun)
eph = load('de421.bsp')

def check_each_camid():
    imgdata = []
    times = []
    fasts = []
    psgs = []
    tles = []
    camids = ['LSC','LSE']
    lists = []
    dicts = []
    
    print('Collecting image information\n')
    for camid in camids:
        d = image_data(camid)
        imgdata.append(d)
        times.append(image_timerange(d))
        fasts.append(h5py.File(f"../fast_20221023{camid}.hdf5", "r"))
        psgs.append(pd.read_pickle(f"../test_data/passages/passages_20221023{camid}.p"))
        tles.append(reduce_tles(camid))
        lists.append([])
        dicts.append({})
    
    # Define observer
    site = get_siteinfo()
    mascara = wgs84.latlon(latitude_degrees=site[0][1], longitude_degrees=site[0][2], elevation_m=site[0][3])
    
    print('Beginning Loop\n')
    for seq in fasts[0]['station']['lstseq'][:]: 
        lstseq = str(seq)

        for data, timerange, camid, fast, starlinks, passages, reslist, resdict, in zip(
            imgdata, times, camids, fasts, tles, psgs, lists, dicts):

            if lstseq not in data.keys():
                #print(f'{lstseq} not in directory')
                continue #this is only needed if not going through ALL images!

            # Set time of image
            midJD = data[lstseq]['midJD']
            ts = load.timescale()
            t = Time(midJD, format='jd')

            # Obtain passages
            idx_reduced = passage_check(midJD, starlinks, passages)

            if idx_reduced is None:
                print(f'{lstseq}: No starlinks passing MASCARA')
                continue

            else:
                print(f'There are {len(idx_reduced)} Starlinks passing overhead {camid} for LSTSEQ={lstseq}')
                # Sun must be low enough below the horizon, otherwise data is not good enough
                observer = mascara + eph['earth']
                sun_pos = observer.at(ts.from_astropy(t)).observe(eph['sun'])
                sun_alt, sun_az, sun_dist = sun_pos.apparent().altaz()
                
                if sun_alt.degrees <= -18.:
                    # Attaining the astrometric solution (depends on FAST file and LSTseq)
                    astro = np.where((fast['astrometry/lstseq'][:] // 50) == (int(lstseq) // 50))[0][0]
                    order = fast['astrometry/x_wcs2pix'][astro].shape[0]-1
                    lst = fast['station/lst'][np.where(fast['station/lstseq'][:]==(fast['astrometry/lstseq'][astro]))[0][0]]
                    nx = data[lstseq]['nx']
                    ny = data[lstseq]['ny']

                    wcspars = { 'crval' : fast['astrometry/crval'][astro].copy(),
                                'crpix' : fast['astrometry/crpix'][astro].copy(),
                                'cdelt' : [0.02148591731740587,0.02148591731740587],
                                'pc'    : fast['astrometry/pc'][astro].copy(),
                                'lst'   : lst }

                    polpars = { 'x_wcs2pix' : fast['astrometry/x_wcs2pix'][astro].copy(),
                                'y_wcs2pix' : fast['astrometry/y_wcs2pix'][astro].copy(),
                                'x_pix2wcs' : fast['astrometry/x_pix2wcs'][astro].copy(),
                                'y_pix2wcs' : fast['astrometry/y_pix2wcs'][astro].copy(),
                                'nx'    : nx,
                                'ny'    : ny,
                                'order' : order }

                    midLST = data[lstseq]['midLST']

                    for idx in idx_reduced:
                        line1 = starlinks[idx][0]
                        line2 = starlinks[idx][1]
                        line3 = starlinks[idx][2] 
                        sat = EarthSatellite(line2, line3, line1, ts)

                        diff = sat - mascara
                        topocentric = diff.at(ts.from_astropy(t))
                        alt, az, dist = topocentric.altaz()

                        # Criteria check: if satellite is >20 degrees above the horizon
                        if alt.degrees >= 20:

                            # Criteria check: if satellite is illuminated
                            illuminated = check_illumination(data, midJD, timerange, sat)

                            if illuminated:
                                ra, dec, radec_dist = topocentric.radec() 
                                radeg = ra._degrees
                                dedeg = dec._degrees
                                ra, dec = j2000_to_equinox(radeg, dedeg, midJD)
                                w = create_wcs(wcspars, midLST)
                                xwcs, ywcs = w.wcs_world2pix(ra, dec, 0)
                                mask = np.isfinite(xwcs) & clip_rectangle(xwcs, ywcs, nx, ny, margin=50)
                                
                                if mask:
                                    print(f"At least one Starlink should be in {camid} FOV")
                                    reslist.append([midJD, lstseq, line2[2:8]])

                    print('\n')



    for fast in fasts:
        fast.close()
        
    for (reslist, resdict, camid) in zip(lists, dicts, camids):
    
        if len(reslist) > 0:
            for sublist in reslist:

                midJD  = sublist[0]
                image  = sublist[1]
                satnum = sublist[2]

                if midJD in resdict:
                    if image in resdict[midJD]:
                        resdict[midJD][image].append(satnum)
                    else:
                        resdict[midJD][image] = [satnum]
                else:
                    resdict[midJD] = {image: [satnum]}

            pickle.dump(resdict, open(f'../result_{camid}.p', 'wb'))
        

In [69]:
%%time
check_each_camid()


Beginning Loop

There are 2 Starlinks passing overhead LSC for LSTSEQ=48506263


There are 2 Starlinks passing overhead LSC for LSTSEQ=48506377
At least one Starlink should be in LSC FOV
At least one Starlink should be in LSC FOV


There are 6 Starlinks passing overhead LSE for LSTSEQ=48506377


There are 2 Starlinks passing overhead LSC for LSTSEQ=48506907


There are 2 Starlinks passing overhead LSC for LSTSEQ=48507687


There are 5 Starlinks passing overhead LSE for LSTSEQ=48507687


There are 1 Starlinks passing overhead LSC for LSTSEQ=48508580


There are 1 Starlinks passing overhead LSC for LSTSEQ=48509045


There are 1 Starlinks passing overhead LSC for LSTSEQ=48509945


There are 3 Starlinks passing overhead LSC for LSTSEQ=48509980


There are 3 Starlinks passing overhead LSC for LSTSEQ=48510542


There are 2 Starlinks passing overhead LSC for LSTSEQ=48510733


There are 2 Starlinks passing overhead LSC for LSTSEQ=48510920
There are 1 Starlinks passing overhead LSC for LSTSEQ=

In [19]:
### REMOVING LSTSEQ TO SEPARATE LIST

# DE421 planetary ephemeris (to get position of Sun)
eph = load('de421.bsp')

def check_each_camid():
    imgdata = []
    times = []
    fasts = []
    psgs = []
    tles = []
    camids = ['LSC','LSE']
    imageIDs = [[]] * len(camids)
    resdicts = [{}] * len(camids)
    
    print('Collecting image information\n')
    for camid in camids:
        d = image_data(camid)
        imgdata.append(d)
        times.append(image_timerange(d))
        fasts.append(h5py.File(f"../fast_20221023{camid}.hdf5", "r"))
        psgs.append(pd.read_pickle(f"../test_data/passages/passages_20221023{camid}.p"))
        tles.append(reduce_tles(camid))
    
    # Define observer
    site = get_siteinfo()
    mascara = wgs84.latlon(latitude_degrees=site[0][1], longitude_degrees=site[0][2], elevation_m=site[0][3])
    
    print('Beginning Loop\n')
    for seq in fasts[0]['station']['lstseq'][:]: 
        lstseq = str(seq)

        for data, timerange, camid, fast, starlinks, passages, imageID, resdict, in zip(
            imgdata, times, camids, fasts, tles, psgs, imageIDs, resdicts):

            if lstseq not in data.keys():
                #print(f'{lstseq} not in directory')
                continue #this is only needed if not going through ALL images!

            # Set time of image
            midJD = data[lstseq]['midJD']
            ts = load.timescale()
            t = Time(midJD, format='jd')

            # Obtain passages
            idx_reduced = passage_check(midJD, starlinks, passages)

            if idx_reduced is None:
                print(f'{lstseq}: No starlinks passing MASCARA')
                continue

            else:
                print(f'There are {len(idx_reduced)} Starlinks passing overhead {camid} for LSTSEQ={lstseq}')
                # Sun must be low enough below the horizon, otherwise data is not good enough
                observer = mascara + eph['earth']
                sun_pos = observer.at(ts.from_astropy(t)).observe(eph['sun'])
                sun_alt, sun_az, sun_dist = sun_pos.apparent().altaz()
                
                if sun_alt.degrees <= -18.:
                    # Attaining the astrometric solution (depends on FAST file and LSTseq)
                    astro = np.where((fast['astrometry/lstseq'][:] // 50) == (int(lstseq) // 50))[0][0]
                    order = fast['astrometry/x_wcs2pix'][astro].shape[0]-1
                    lst = fast['station/lst'][np.where(fast['station/lstseq'][:]==(fast['astrometry/lstseq'][astro]))[0][0]]
                    nx = data[lstseq]['nx']
                    ny = data[lstseq]['ny']

                    wcspars = { 'crval' : fast['astrometry/crval'][astro].copy(),
                                'crpix' : fast['astrometry/crpix'][astro].copy(),
                                'cdelt' : [0.02148591731740587,0.02148591731740587],
                                'pc'    : fast['astrometry/pc'][astro].copy(),
                                'lst'   : lst }

                    polpars = { 'x_wcs2pix' : fast['astrometry/x_wcs2pix'][astro].copy(),
                                'y_wcs2pix' : fast['astrometry/y_wcs2pix'][astro].copy(),
                                'x_pix2wcs' : fast['astrometry/x_pix2wcs'][astro].copy(),
                                'y_pix2wcs' : fast['astrometry/y_pix2wcs'][astro].copy(),
                                'nx'    : nx,
                                'ny'    : ny,
                                'order' : order }

                    midLST = data[lstseq]['midLST']

                    for idx in idx_reduced:
                        line1 = starlinks[idx][0]
                        line2 = starlinks[idx][1]
                        line3 = starlinks[idx][2] 
                        sat = EarthSatellite(line2, line3, line1, ts)

                        diff = sat - mascara
                        topocentric = diff.at(ts.from_astropy(t))
                        alt, az, dist = topocentric.altaz()

                        # Criteria check: if satellite is >20 degrees above the horizon
                        if alt.degrees >= 20:

                            # Criteria check: if satellite is illuminated
                            illuminated = check_illumination(data, midJD, timerange, sat)

                            if illuminated:
                                ra, dec, radec_dist = topocentric.radec() 
                                radeg = ra._degrees
                                dedeg = dec._degrees
                                ra, dec = j2000_to_equinox(radeg, dedeg, midJD)
                                w = create_wcs(wcspars, midLST)
                                xwcs, ywcs = w.wcs_world2pix(ra, dec, 0)
                                mask = np.isfinite(xwcs) & clip_rectangle(xwcs, ywcs, nx, ny, margin=50)
                                
                                if mask:
                                    print(f"{line1} should be in {camid} FOV")
                                    if lstseq not in imageID:
                                        imageID.append(lstseq)
                                    if midJD not in resdict:
                                        resdict[midJD] = [line2[2:8]]
                                    else:
                                        resdict[midJD].append(line2[2:8])

                    print('\n')



    for fast in fasts:
        fast.close()
        
    for (imageID, resdict, camid) in zip(imageIDs, resdicts, camids):
    
        if len(imageID) > 0:
            pickle.dump(resdict, open(f'../result_{camid}.p', 'wb'))
            #np.savetxt(f'../lstseqs_{camid}.txt', imageID, fmt='%s')
            np.save(f'../lstseqs_{camid}.npy', imageID)
            
            
## NO -- we need to encode some lstseq info. Or else we'd be running loads of starlinks per image that aren't there.



In [20]:
%%time
check_each_camid()


Beginning Loop

There are 2 Starlinks passing overhead LSC for LSTSEQ=48506263


There are 2 Starlinks passing overhead LSC for LSTSEQ=48506377
STARLINK-2631 should be in LSC FOV
STARLINK-3641 should be in LSC FOV


There are 6 Starlinks passing overhead LSE for LSTSEQ=48506377


There are 2 Starlinks passing overhead LSC for LSTSEQ=48506907


There are 2 Starlinks passing overhead LSC for LSTSEQ=48507687


There are 5 Starlinks passing overhead LSE for LSTSEQ=48507687


There are 1 Starlinks passing overhead LSC for LSTSEQ=48508580


There are 1 Starlinks passing overhead LSC for LSTSEQ=48509045


There are 1 Starlinks passing overhead LSC for LSTSEQ=48509945


There are 3 Starlinks passing overhead LSC for LSTSEQ=48509980


There are 3 Starlinks passing overhead LSC for LSTSEQ=48510542


There are 2 Starlinks passing overhead LSC for LSTSEQ=48510733


There are 2 Starlinks passing overhead LSC for LSTSEQ=48510920
There are 1 Starlinks passing overhead LSC for LSTSEQ=48510957
CPU tim

In [33]:
passages = pd.read_pickle("../test_data/passages/passages_20221023LSC.p")
X = pd.read_pickle("../result_LSC.p")

In [39]:
# Reduce passages file to selection pool: (THIS IS CAMID DEPENDENT!)

# X = selection pool
# Y = passages file reduced to midJDs found in X
# Z = Y file reduced such that each midJD has only the satnums found in X 

# Firstly reduce midJDs
Y = {}
Y = {midJD: data for midJD, data in passages.items() if midJD in list(X)}

# Now reduce satnums in each midJD
Z = {}
for midJD in X:
    Z[midJD] = {}
    for satnum in X[midJD]:
        if satnum in Y[midJD]:
            Z[midJD][satnum] = Y[midJD][satnum]

In [40]:
# TEST

for key in X.keys():
    print(X[key])
    
for key in Z.keys():
    print(list(Z[key]))

['48366U', '51788U']
['48366U', '51788U']


In [130]:
#### TEST

lstseqs = glob.glob(f'../selection_pool/lstseqs_*.npy')
pools = glob.glob(f'../selection_pool/pool_*.p')

print(lstseqs[3])
print(pools[3])

pool_LSC = pd.read_pickle(pools[3])
lstseq_LSC = np.load(lstseqs[3])

../selection_pool/lstseqs_LSC.npy
../selection_pool/pool_LSC.p


## V5 -- we need to encode some lstseq info. Or else we'd be running loads of starlinks per image that aren't there

In [25]:
resdict = {}

line1 = 'U'
lstseq1 = '100'

line2 = 'N'
lstseq2 = '200'

for midJD in [10, 10, 20]:
    if midJD not in resdict:
        resdict[midJD] = {}
        resdict[midJD][line1] = lstseq1
    else:
        resdict[midJD][line2] = lstseq2

In [30]:
for key in resdict.keys():
    print(resdict[key].keys())

dict_keys(['U', 'N'])
dict_keys(['U'])


In [71]:
# DE421 planetary ephemeris (to get position of Sun)
eph = load('de421.bsp')

def check_each_camid():
    imgdata = []
    times = []
    fasts = []
    psgs = []
    tles = []
    camids = ['LSC','LSE']
    pooldicts = [{}] * len(camids)
    
    print('Collecting image information\n')
    for camid in camids:
        d = image_data(camid)
        imgdata.append(d)
        times.append(image_timerange(d))
        fasts.append(h5py.File(f"../fast_20221023{camid}.hdf5", "r"))
        psgs.append(pd.read_pickle(f"../test_data/passages/passages_20221023{camid}.p"))
        tles.append(reduce_tles(camid))
    
    # Define observer
    site = get_siteinfo()
    mascara = wgs84.latlon(latitude_degrees=site[0][1], longitude_degrees=site[0][2], elevation_m=site[0][3])
    
    print('Beginning Loop\n')
    for seq in fasts[0]['station']['lstseq'][:]: 
        lstseq = str(seq)

        for data, timerange, camid, fast, starlinks, passages, pool, in zip(
            imgdata, times, camids, fasts, tles, psgs, pooldicts):

            if lstseq not in data.keys():
                #print(f'{lstseq} not in directory')
                continue #this is only needed if not going through ALL images!

            # Set time of image
            midJD = data[lstseq]['midJD']
            ts = load.timescale()
            t = Time(midJD, format='jd')

            # Obtain passages
            idx_reduced = passage_check(midJD, starlinks, passages)

            if idx_reduced is None:
                print(f'{lstseq}: No starlinks passing MASCARA')
                continue

            else:
                print(f'There are {len(idx_reduced)} Starlinks passing overhead {camid} for LSTSEQ={lstseq}')
                # Sun must be low enough below the horizon, otherwise data is not good enough
                observer = mascara + eph['earth']
                sun_pos = observer.at(ts.from_astropy(t)).observe(eph['sun'])
                sun_alt, sun_az, sun_dist = sun_pos.apparent().altaz()
                
                if sun_alt.degrees <= -18.:
                    # Attaining the astrometric solution (depends on FAST file and LSTseq)
                    astro = np.where((fast['astrometry/lstseq'][:] // 50) == (int(lstseq) // 50))[0][0]
                    order = fast['astrometry/x_wcs2pix'][astro].shape[0]-1
                    lst = fast['station/lst'][np.where(fast['station/lstseq'][:]==(fast['astrometry/lstseq'][astro]))[0][0]]
                    nx = data[lstseq]['nx']
                    ny = data[lstseq]['ny']

                    wcspars = { 'crval' : fast['astrometry/crval'][astro].copy(),
                                'crpix' : fast['astrometry/crpix'][astro].copy(),
                                'cdelt' : [0.02148591731740587,0.02148591731740587],
                                'pc'    : fast['astrometry/pc'][astro].copy(),
                                'lst'   : lst }

                    polpars = { 'x_wcs2pix' : fast['astrometry/x_wcs2pix'][astro].copy(),
                                'y_wcs2pix' : fast['astrometry/y_wcs2pix'][astro].copy(),
                                'x_pix2wcs' : fast['astrometry/x_pix2wcs'][astro].copy(),
                                'y_pix2wcs' : fast['astrometry/y_pix2wcs'][astro].copy(),
                                'nx'    : nx,
                                'ny'    : ny,
                                'order' : order }

                    midLST = data[lstseq]['midLST']

                    for idx in idx_reduced:
                        line1 = starlinks[idx][0]
                        line2 = starlinks[idx][1]
                        line3 = starlinks[idx][2] 
                        sat = EarthSatellite(line2, line3, line1, ts)

                        diff = sat - mascara
                        topocentric = diff.at(ts.from_astropy(t))
                        alt, az, dist = topocentric.altaz()

                        # Criteria check: if satellite is >20 degrees above the horizon
                        if alt.degrees >= 20:

                            # Criteria check: if satellite is illuminated
                            illuminated = check_illumination(data, midJD, timerange, sat)

                            if illuminated:
                                ra, dec, radec_dist = topocentric.radec() 
                                radeg = ra._degrees
                                dedeg = dec._degrees
                                ra, dec = j2000_to_equinox(radeg, dedeg, midJD)
                                w = create_wcs(wcspars, midLST)
                                xwcs, ywcs = w.wcs_world2pix(ra, dec, 0)
                                mask = np.isfinite(xwcs) & clip_rectangle(xwcs, ywcs, nx, ny, margin=50)
                                
                                if mask:
                                    print(f"{line1} should be in {camid} FOV")
                                    if midJD not in pool:
                                        pool[midJD] = {}
                                        pool[midJD][line2[2:8]] = lstseq
                                    else:
                                        pool[midJD][line2[2:8]] = lstseq

                    print('\n')



    for fast in fasts:
        fast.close()
        
    for (pool, camid) in zip(pooldicts, camids):
    
        if len(pool) > 0:
            pickle.dump(pool, open(f'../pool_{camid}.p', 'wb'))            



In [72]:
%%time
check_each_camid()


Beginning Loop

There are 2 Starlinks passing overhead LSC for LSTSEQ=48506263


There are 2 Starlinks passing overhead LSC for LSTSEQ=48506377
STARLINK-2631 should be in LSC FOV
STARLINK-3641 should be in LSC FOV


There are 6 Starlinks passing overhead LSE for LSTSEQ=48506377


There are 2 Starlinks passing overhead LSC for LSTSEQ=48506907


There are 2 Starlinks passing overhead LSC for LSTSEQ=48507687


There are 5 Starlinks passing overhead LSE for LSTSEQ=48507687


There are 1 Starlinks passing overhead LSC for LSTSEQ=48508580


There are 1 Starlinks passing overhead LSC for LSTSEQ=48509045


There are 1 Starlinks passing overhead LSC for LSTSEQ=48509945


There are 3 Starlinks passing overhead LSC for LSTSEQ=48509980


There are 3 Starlinks passing overhead LSC for LSTSEQ=48510542


There are 2 Starlinks passing overhead LSC for LSTSEQ=48510733


There are 2 Starlinks passing overhead LSC for LSTSEQ=48510920
There are 1 Starlinks passing overhead LSC for LSTSEQ=48510957
CPU tim

In [73]:
resdicts = glob.glob('../pool_*.p')
resdicts

['../pool_LSE.p', '../pool_LSC.p']

In [74]:
# NEW
pool_LSE = pd.read_pickle(resdicts[0])
pool_LSC = pd.read_pickle(resdicts[1])
print(pool_LSE.items())
print(pool_LSC.items())

dict_items([(2459876.52443372, {'48366U': '48506377', '51788U': '48506377'})])
dict_items([(2459876.52443372, {'48366U': '48506377', '51788U': '48506377'})])


In [70]:
# OLD
# pool_LSE = pd.read_pickle('../test/result_LSE.p')
pool_LSC = pd.read_pickle('../test/result_LSC.p')
# print(pool_LSE.items())
print(pool_LSC.items())

dict_items([(2459876.52443372, {'48506377': ['48366U', '51788U']})])


In [65]:
result

['48506377LSC STARLINK-2631 48366U', '48506377LSC STARLINK-3641 51788U']

In [138]:
#### TEST

lstseqs = glob.glob(f'../selection_pool/lstseqs_*.npy')
pools = glob.glob(f'../selection_pool/pool_*.p')

print(lstseqs[3])
print(pools[3])

pool_LSC = pd.read_pickle(pools[3])
lstseq_LSC = np.load(lstseqs[3])

../selection_pool/lstseqs_LSC.npy
../selection_pool/pool_LSC.p


In [139]:
passages = pd.read_pickle("../test_data/passages/passages_20221023LSC.p")

In [150]:
# Reduce passages file to selection pool: (THIS IS CAMID DEPENDENT!)

# X = selection pool
# Y = passages file reduced to midJDs found in X
# Z = Y file reduced such that each midJD has only the satnums found in X 

# Firstly reduce midJDs
Y = {}
for midJD, data in passages.items():
    if midJD in pool_LSE:
        Y[midJD] = data
        
# Y = {midJD: data for midJD, data in passages.items() if midJD in list(pool_LSE)}

# Now reduce satnums in each midJD
Z = {}
for midJD in Y:
    Z[midJD] = {}
    for satnum in pool_LSC[midJD]:
        if satnum in Y[midJD]:
            Z[midJD][satnum] = Y[midJD][satnum]

In [153]:
pool_LSE[2459876.528127297]

['44932U', '47789U', '49422U']

In [147]:
list(Y[2459876.528127297])

['00271U',
 '00745U',
 '01358U',
 '02766U',
 '04598U',
 '07400U',
 '10011U',
 '10150U',
 '10167U',
 '10573U',
 '10722U',
 '10998U',
 '11054U',
 '11550U',
 '11888U',
 '12817U',
 '14049U',
 '14264U',
 '14592U',
 '14735U',
 '14796U',
 '15055U',
 '15261U',
 '15264U',
 '15266U',
 '15698U',
 '15702U',
 '15873U',
 '16404U',
 '17147U',
 '18356U',
 '18881U',
 '19501U',
 '19505U',
 '19592U',
 '20672U',
 '20843U',
 '21011U',
 '21029U',
 '21221U',
 '21552U',
 '21588U',
 '21637U',
 '21640U',
 '21765U',
 '21853U',
 '21858U',
 '22512U',
 '22719U',
 '22724U',
 '22988U',
 '23043U',
 '23045U',
 '23176U',
 '23185U',
 '23229U',
 '23292U',
 '23396U',
 '23429U',
 '23435U',
 '23511U',
 '23651U',
 '23725U',
 '23815U',
 '23943U',
 '24768U',
 '24960U',
 '25253U',
 '25774U',
 '25839U',
 '25900U',
 '25933U',
 '26050U',
 '26065U',
 '26535U',
 '26564U',
 '26577U',
 '26738U',
 '26964U',
 '27525U',
 '27576U',
 '27704U',
 '27792U',
 '27919U',
 '27938U',
 '28112U',
 '28129U',
 '28156U',
 '28256U',
 '28512U',
 '28820U',

In [135]:
i = 0
for jd in pool_LSC.keys():
    if jd in passages.keys():
        i += 1

In [136]:
len(pool_LSC)

21

In [137]:
i

2